# Load small dandiset

author: laquitainesteeve@gmail.com

purpose: create small demo dataset (neuropixels evoked 20 KHz)

# Setup 

Activate dandi virtual environment (envs/dandi.yml)

```bash
python -m ipykernel install --user --name dandi --display-name "dandi"
```

In [4]:
%%time 

# import python packages
from dandi.dandiapi import DandiAPIClient
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface
spikeinterface.__version__
from time import time

CPU times: user 17 μs, sys: 0 ns, total: 17 μs
Wall time: 19.1 μs


## Load dandiset

In [7]:
%%time

# load dandiset (npx, evoked, 20Khz)
dandiset_id = '001250'
filepath = 'sub-002-fitted/sub-002-fitted_ecephys.nwb'

# get tthe file path on S3
with DandiAPIClient() as client:
    asset = client.get_dandiset(dandiset_id, 'draft').get_asset_by_path(filepath)
    s3_path = asset.get_content_url(follow_redirects=1, strip_query=True)
print("s3_path:", s3_path)

# get RecordingExtractor
recording = se.NwbRecordingExtractor(file_path=s3_path, stream_mode="remfile")
print("\nRecording:", recording)

s3_path: https://dandiarchive.s3.amazonaws.com/blobs/9d6/6ed/9d66ed40-af31-43aa-b4ba-246d2206dcad

Recording: NwbRecordingExtractor: 384 channels - 20.0kHz - 1 segments - 72,359,964 samples 
                       3,618.00s (1.00 hours) - float32 dtype - 103.51 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/9d6/6ed/9d66ed40-af31-43aa-b4ba-246d2206dcad
CPU times: user 36.5 ms, sys: 5.21 ms, total: 41.7 ms
Wall time: 627 ms


In [21]:
%%time 

# Re-wire probe because it is not always recognized by kilosort4
# and unit-test
probe = recording.get_probe()
recording.set_probe(probe)
# assert recording.get_probe(), "A probe should be attached to this recording"
assert "layers" in recording.get_property_keys(), "RecordingExtractor should contain layer property"

## Make the small dataset